In [2]:
from pandas import DataFrame

data = DataFrame({
    "aks_number": ["420A1", "420A1", "420B1", "420C1"],
    "other_01": [1, 2, 3, 4],
    "other_02": ["A", "B", "C", "D"],
})

In [4]:
data

,aks_number,other_01,other_02
0,420A1,1,A
1,420A1,2,B
2,420B1,3,C
3,420C1,4,D


# Get duplicate lines

In [5]:
data.loc[data.duplicated("aks_number")]

,aks_number,other_01,other_02
1,420A1,2,B


# Get non-duplicate lines

In [10]:
data.loc[~data.duplicated("aks_number")]

,aks_number,other_01,other_02
0,420A1,1,A
2,420B1,3,C
3,420C1,4,D
